In [2]:
import numpy as np
from scipy.spatial import distance
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import sys

name = "dark_mat"

location_grid_coords = "../data/FIRE/grid-coords.npy"
location_non_empty_voxel_coords = "../data/FIRE/non-empty-coords-"+name+".npy"
# name = location_non_empty_voxel_coords[30:][0:location_non_empty_voxel_coords[30:].rfind(".")]
location_save_grid_ids = "../data/FIRE/grid-ids--"+name+".npy"

grid_c = np.load(location_grid_coords)
print(f">> loaded in voxel grid coordinates from {location_grid_coords}")

non_empty_points = np.load(location_non_empty_voxel_coords)
print(f">> loaded in non-empty voxel coordinates from {location_non_empty_voxel_coords}")

print(f">> {name}")



>> loaded in voxel grid coordinates from ../data/FIRE/grid-coords.npy
>> loaded in non-empty voxel coordinates from ../data/FIRE/non-empty-coords-dark_mat.npy
>> dark_mat


In [46]:
start = 10
end = 20


points_offset_index = 0
distances = distance.cdist(grid_c[start:end], non_empty_points[points_offset_index:points_offset_index+5])
current_minimum_distances = np.min(distances, axis=1)
current_minimum_indexes = np.argmin(distances, axis=1) + points_offset_index

print(distances)
print(current_minimum_distances)
print(current_minimum_indexes)

print("----------------")

points_offset_index = points_offset_index + 5
distances = distance.cdist(grid_c[start:end], non_empty_points[points_offset_index:points_offset_index+5])
proposed_minimum_distances = np.min(distances, axis=1)
proposed_minimum_indexes = np.argmin(distances, axis=1) + points_offset_index

new_minimum_distances_boolean = proposed_minimum_distances < current_minimum_distances

current_minimum_distances[new_minimum_distances_boolean] = proposed_minimum_distances[new_minimum_distances_boolean]
current_minimum_indexes[new_minimum_distances_boolean] = proposed_minimum_indexes[new_minimum_distances_boolean]



print(distances)
print(current_minimum_indexes)
# print(proposed_minimum_distances)

print(current_minimum_indexes[proposed_minimum_distances < current_minimum_distances])



print(np.argmin(distances, axis=1)[proposed_minimum_distances < current_minimum_distances] + points_offset_index)

[[ 4.84375     6.8359375  11.8359375   3.12524413  6.32824556]
 [ 4.8046875   6.796875   11.796875    3.08618472  6.28918381]
 [ 4.765625    6.7578125  11.7578125   3.04712539  6.25012207]
 [ 4.7265625   6.71875    11.71875     3.00806614  6.21106034]
 [ 4.6875      6.6796875  11.6796875   2.96900698  6.17199861]
 [ 4.6484375   6.640625   11.640625    2.92994791  6.1329369 ]
 [ 4.609375    6.6015625  11.6015625   2.89088892  6.0938752 ]
 [ 4.5703125   6.5625     11.5625      2.85183004  6.05481351]
 [ 4.53125     6.5234375  11.5234375   2.81277125  6.01575182]
 [ 4.4921875   6.484375   11.484375    2.77371257  5.97669015]]
[3.12524413 3.08618472 3.04712539 3.00806614 2.96900698 2.92994791
 2.89088892 2.85183004 2.81277125 2.77371257]
[3 3 3 3 3 3 3 3 3 3]
----------------
[[12.6953726  13.75005549 17.81254283  6.99262394 10.58622578]
 [12.65631028 13.71099314 17.77348043  6.95356389 10.54716435]
 [12.61724797 13.6719308  17.73441802  6.91450387 10.50810292]
 [12.57818566 13.63286846 17

In [ ]:

def process_interval(start, end, points_offset_start, points_offset_end):
    
    offset_index = points_offset_start
    distances = distance.cdist(grid_c[start:end], non_empty_points[points_offset_index:points_offset_index+5])
    proposed_minimum_distances = np.min(distances, axis=1)

In [47]:
# distances

In [48]:
# np.argmin(distances, axis=1)

In [49]:
# np.min(distance.cdist(grid_c[start:end], non_empty_points[0:9]), axis=1)

In [4]:
# Parameters
batch_size = 100

# Function to calculate distances and update grid_ids for a given interval
def process_interval(start, end):
    grid_ids[start:end] = np.argmin(distance.cdist(grid_c[start:end], non_empty_points), axis=1)

# Create an array to store grid_ids
grid_ids = np.empty(len(grid_c), dtype=int)

# Create batches
batch_counter = np.arange(batch_size, len(grid_c), batch_size)
batch_counter[-1] = len(grid_c)

In [18]:


np.argmin(distance.cdist(grid_c[start:end], non_empty_points[9:20]), axis=1)

array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [12]:
# for interval_min, interval_max in zip([0] + list(batch_counter), batch_counter):
#     print(interval_min, interval_max)

In [ ]:


# Process batches concurrently
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_interval, interval_min, interval_max): (interval_min, interval_max)
               for interval_min, interval_max in zip([0] + list(batch_counter), batch_counter)}

    for future in concurrent.futures.as_completed(futures):
        interval_min, interval_max = futures[future]
        print(f"Working on batch [{interval_min}-{interval_max}] out of {len(grid_c)}, so {interval_max/len(grid_c)*100}% done.", end="\r")
        # print(f"Processed interval [{interval_min}:{interval_max}]")


np.save(location_save_grid_ids, grid_ids)
